# Projekt IUM. Etap 1. Iteracja 1.
## Andrii Gamalii, Wiktor Topolski 
### Zadanie “Większość serwisów udostępniających muzykę czy filmiki, poleca coś swoim użytkownikom. Przyszedł czas, abyśmy zaczęli robić tak samo”

i)
definicję problemu biznesowego, zdefiniowanie zadania/zadań modelowania i wszystkich założeń, zaproponowania kryteriów sukcesu),
ii)
analizę danych z perspektywy realizacji tych zadań (trzeba ocenić, czy dostarczone dane są wystarczające – może czegoś brakuje, może coś trzeba poprawić, domagać się innych danych, …),

# Definicja problemu biznesowego, definicja zadania modelowania i wszystkich założeń, propozycja kryteriów sukcesu

## Problem biznesowy
Polecenie utworów użytkownikom serwisu do strumieniowania muzyki

## Zadanie modelowania
Generacja rekomendacji na podstawie logów sesji użytkowników
## Założenia
Rekomendacje pojawiają się na ekranie startowym  
Rekomendujemy 10 utworów

## Biznesowe kryteria sukcesu
Użytkownik wybierze 1 z 10 zaproponowanych utworów w 75% sytuacji

# Analiza danych

In [49]:
import pandas as pd

In [50]:
# Wczytywanie danych z plików

artists_df_orig = pd.read_json("./data/artists.jsonl", lines=True)
sessions_df_orig = pd.read_json("./data/sessions.jsonl", lines=True)
tracks_df_orig = pd.read_json("./data/tracks.jsonl", lines=True)
users_df_orig = pd.read_json("./data/users.jsonl", lines=True)

## Wykonawcy

In [90]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami

artists_df = artists_df_orig.copy(deep=True)
print("Liczba rekordów", len(artists_df))
print("Liczba unikalnych rekordów po id", len(artists_df[['id']].drop_duplicates()))
print("Wyniki: ")
for c in artists_df.columns:
    incomplete_rows = artists_df[artists_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

Liczba rekordów 1667
Liczba unikalnych rekordów po id 1597
Wyniki: 
id 0
name 0
genres 81


### Wnioski
Niektóre rekordy mają id = -1, to powoduje, że są nieużyteczne
- id i name: nie mają brakujących wartości
- genres: braki w 81 rekordach, można uzupełnić na podstawie piosenek

## Sesje

In [86]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami

sessions_df = sessions_df_orig.copy(deep=True)
print("Liczba rekordów", len(sessions_df))
print("Liczba unikalnych rekordów", len(sessions_df.drop_duplicates()))
print("Wyniki: ")
for c in sessions_df.columns:
    incomplete_rows = sessions_df[sessions_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

Liczba rekordów 3173490
Liczba unikalnych rekordów po id 3173383
Wyniki: 
session_id 0
timestamp 0
user_id 158691
track_id 565752
event_type 158853


Brakujących danych jest dużo...
Trochę powtarzających się rekordów (107). Je można bez żadnych konsekwencji usunąć

In [55]:
print("Unikalne wartości event_type", sessions_df['event_type'].unique())

Unikalne wartości event_type ['play' 'advertisment' 'like' 'skip' None]


Niektóre rekordy zawierają wartość 'advertisment' w kolumnie event_type, są one nam nie potrzebne
Możemy bez konsekwencji je usunąć

In [59]:
# liczba rekordów z event_type = 'advertisment'

sessions_df = sessions_df_orig.copy(deep=True)
print(len(sessions_df[sessions_df['event_type'].isin(['advertisment'])]))

407132


Jest tego sporo...

In [60]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami wyłączając takie z event_type = 'advertisment'

sessions_df = sessions_df_orig.copy(deep=True)
# Usuwanie event_type = 'advertisment'
sessions_df = sessions_df[sessions_df['event_type'].isin(['skip', 'like', None, 'play'])]

print("Liczba rekordów", len(sessions_df))
print("Wyniki: ")
for c in sessions_df.columns:
    incomplete_rows = sessions_df[sessions_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

Liczba rekordów 2766358
Wyniki: 
session_id 0
timestamp 0
user_id 138255
track_id 158620
event_type 158853


In [61]:
# liczba rekordów z brakującymi track_id lub event_type

print(len(sessions_df[sessions_df[["track_id", "event_type"]].isnull().any(axis=1)]))

289038


### Wnioski
- user_id: da się wywnioskować na podstawie innego rekordu o takim samym id sesji
- track_id i event_type: te braki powodują bezużyteczność danego rekordu oraz nieciągłość sekwencji akcji użytkownika w sesji, co powoduje, że dane dla takich sesji są mniej użyteczne. Jest to około 10% danych

## Utwory

In [87]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami

tracks_df = tracks_df_orig.copy(deep=True)
print("Liczba rekordów", len(tracks_df))
print("Liczba unikalnych rekordów", len(tracks_df.drop_duplicates()))
print("Wyniki: ")
for c in tracks_df.columns:
    incomplete_rows = tracks_df[tracks_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

Liczba rekordów 22412
Liczba unikalnych rekordów 22412
Wyniki: 
id 1161
name 1146
popularity 1184
duration_ms 0
explicit 0
id_artist 1073
release_date 0
danceability 0
energy 0
key 0
loudness 0
speechiness 0
acousticness 0
instrumentalness 0
liveness 0
valence 0
tempo 0


### Wnioski
- bez id nie jesteśmy w stanie użyć danego utworu
- bez name nie możemy przedstawić utworu użytkownikowi
- braki w popularity nie skutkują bezużytecznością danego rekordu
- braki w id_artist powodują problemy z powiązaniem piosenek tego samego artysty między sobą 

## Użytkownicy

In [89]:
# Sprawdzenie ile jest rekordów z brakującymi wartościami

users_df = users_df_orig.copy(deep=True)
print("Liczba rekordów", len(users_df))
print("Liczba unikalnych rekordów", len(users_df["user_id"].drop_duplicates()))
print("Wyniki: ")
for c in users_df.columns:
    incomplete_rows = users_df[users_df[[c]].isnull().any(axis=1)]
    print(c, len(incomplete_rows))

Liczba rekordów 1000
Liczba unikalnych rekordów 1000
Wyniki: 
user_id 0
name 0
city 0
street 0
favourite_genres 48
premium_user 43
id 945


In [65]:
# Sprawdzenie ile rekordów mają brakujące wartości w favourite_genres albo premium_user

print(len(users_df[users_df[["favourite_genres", "premium_user"]].isnull().any(axis=1)]))

90


### Wnioski
- w 90 liniach brakuje albo favourite_genres albo premium_user, ale to nie powinno przeszkadzać w realizacji zadania rekomendacji.

- nie wiadomo co znaczy atrybut id, który pojawia się 55 razy i zawsze ma wartość -1.

# WNIOSKI

- Wykonawcy: 
- - potrzebne jest wprowadzenie poprawnych id dla rekordów z id = -1
- Sesje:
- - potrzebne dane z uzupełnionymi track_id i event_type
- Utwory:
- - potrzebne uzupełnienie id i name dla utworów oraz id_artist
- Użytkownicy:
- - potrzebna informacja o znaczeniu atrybutu id
